## Imports

In [ ]:
try:
    import uuid
    from time import sleep

    from datetime import datetime
    import elasticsearch
    from elasticsearch import Elasticsearch
    
    import json
    from tqdm import tqdm
    import os
    import sys
    
    from elasticsearch import helpers
    print("Loaded successful.")
except Exception as E:
    print("Load failed: {} ".format(e))

In [ ]:
ENDPOINT = "http://localhost:9200/"
es = Elasticsearch(timeout=600, hosts=ENDPOINT)
es.ping()

### Reading Data

In [ ]:
with open("test_laws.txt", encoding='utf-8') as f:
    all_laws = f.readlines()
all_laws = [x.strip() for x in all_laws]

In [ ]:
from pandas import DataFrame
df = DataFrame(all_laws)

In [ ]:
def generator(all_laws):
    for c, line in enumerate(all_laws):
        yield {
            '_index': 'mylkfirst',
            '_type' : '_doc',
            '_id': c,
            '_source': {
                line
            }
        }
    raise StopIteration

In [ ]:
mycustom = generator(all_laws)

In [ ]:
mycustom

In [ ]:
next(mycustom)

### Settings

In [ ]:
Settings = {
    "settings" : {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings" : {
        "properties": {
            "Label": { "type": "keyword" }
        }
    }
}

In [ ]:
Indexname = ''
my = es.indices.create(index='mylkfirst', ignore=[400,404], body=Settings)

In [ ]:
my

### Upload Data to Elasticsearch

In [ ]:
try:
    result = helpers.bulk(es, generator(all_laws))
    print("Success")
except Exception as e:
    pass

In [ ]:
result = helpers.bulk(es, generator(all_laws))

In [46]:
def get_data_from_text_file(self):
    return [l.strip() for l in open(str(self), encoding="utf8", errors='ignore')]

docs = get_data_from_text_file("laws.json")

In [47]:
# define an empty list for the Elasticsearch docs
doc_list = []

for num, doc in enumerate(docs):

    # catch any JSON loads() errors
    try:
        doc = doc.replace("True", "true")
        doc = doc.replace("False", "false")

        dict_doc = json.loads(doc)

        dict_doc["timestamp"] = datetime.now()

        dict_doc["_id"] = num

        doc_list += [dict_doc]
    except json.decoder.JSONDecodeError as err:
        print ("ERROR for num:", num, "-- JSONDecodeError:", err, "for doc:", doc)

print ("Dict docs length:", len(doc_list))

try:
    print ("\nAttempting to index the list of docs using helpers.bulk()")

    resp = helpers.bulk(
        es,
        doc_list,
        index = "some_index",
        doc_type = "_doc"
    )
    
    print ("helpers.bulk() RESPONSE:", resp)
    print ("helpers.bulk() RESPONSE:", json.dumps(resp, indent=4))

except Exception as err:
    # print any errors returned 
    print("Elasticsearch helpers.bulk() ERROR:", err)

Dict docs length: 18

Attempting to index the list of docs using helpers.bulk()
helpers.bulk() RESPONSE: (18, [])
helpers.bulk() RESPONSE: [
    18,
    []
]
